In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
from urllib.request import urlopen
import json
import numpy as np
import pandas as pd
from datetime import datetime, date
from tqdm.notebook import tqdm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from app.utils.fetch_data import fred_fred, investing_api, alpha_vantage_api, FMP

In [11]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)
    
key = keys['financial_modeling_prep']

In [3]:
today = date.today().strftime("%Y-%m-%d")
today

'2022-07-19'

In [12]:
total_stock = investing_api('stock', 'AAPL', '2016-01-01', today)

2022-07-19 23:10:36,809 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: AAPL, from 01/01/2016 to 19/07/2022


In [13]:
total_stock.Date

0      2016-01-04
1      2016-01-05
2      2016-01-06
3      2016-01-07
4      2016-01-08
          ...    
1643   2022-07-13
1644   2022-07-14
1645   2022-07-15
1646   2022-07-18
1647   2022-07-19
Name: Date, Length: 1648, dtype: datetime64[ns]

#### Fetch insider trading data

In [4]:
fmp = FMP()

In [8]:
insider = fmp.get_historical_insider_trade_ticker("AAPL", num_pages=11)

In [14]:
insider.transactionDate.astype(datetime64)

NameError: name 'datetime64' is not defined

In [10]:
insider.head(2)

,symbol,filingDate,transactionDate,reportingCik,transactionType,securitiesOwned,companyCik,reportingName,typeOfOwner,acquistionOrDisposition,formType,securitiesTransacted,price,securityName,link
0,AAPL,2022-05-13 18:31:28,2022-05-06,0001182047,G-Gift,34990.0,0000320193,BELL JAMES A,director,D,4,1276.0,0.00,Common Stock,https://www.sec.gov/Archives/edgar/data/000032...
1,AAPL,2022-05-06 18:30:57,2022-05-04,0001462356,S-Sale,452334.0,0000320193,Adams Katherine L.,"officer: SVP, GC and Secretary",D,4,2292.0,165.95,Common Stock,https://www.sec.gov/Archives/edgar/data/000032...


In [ ]:
# only common stocks.
# Restricted stock units are special kind of stocks that is given to the employee under certain conditions

In [111]:
insider['transactionDate_Y_M'] = insider.transactionDate.str[:7]

In [112]:
volume_by_AD = insider.groupby(['transactionDate_Y_M', 'acquistionOrDisposition']).agg({"securitiesTransacted":"sum"}).reset_index()
volume_by_AD.head(2)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
0,2016-04,A,219268.0
1,2016-04,D,352435.0


In [116]:
dates = volume_by_AD.transactionDate_Y_M.drop_duplicates()

acquisition = volume_by_AD.loc[volume_by_AD.acquistionOrDisposition=='A']
acquisition = pd.merge(dates, acquisition, how='left')
acquisition['acquistionOrDisposition'] = acquisition['acquistionOrDisposition'].fillna('A')
acquisition['securitiesTransacted'] = acquisition['securitiesTransacted'].fillna(0)

disposition = volume_by_AD.loc[volume_by_AD.acquistionOrDisposition=='D']
disposition = pd.merge(dates, disposition, how='left')
disposition['acquistionOrDisposition'] = disposition['acquistionOrDisposition'].fillna('D')
disposition['securitiesTransacted'] = disposition['securitiesTransacted'].fillna(0)

In [117]:
disposition.head(2)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
0,2016-04,D,352435.0
1,2016-05,D,519946.0


In [118]:
acquisition.tail(3)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
58,2022-03,A,13480.0
59,2022-04,A,620558.0
60,2022-05,A,0.0


In [122]:
fig = go.Figure()

trace1 = go.Bar(x=dates,
                y=acquistion.securitiesTransacted,
                name="Acquisition",
                marker_color='indianred')
trace2 = go.Bar(x=dates,
                y=disposition.securitiesTransacted,
                marker_color='blue',
                name="Disposition")

fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(barmode='group')

In [66]:
list(insider.link)[0]

'https://www.sec.gov/Archives/edgar/data/0000320193/000032019322000063/0000320193-22-000063-index.htm'

In [ ]:
trace = go.Bar(x=data[x],
               y=data[y],
               name=name,
               yaxis=yaxis)

In [ ]:
# disposition and acquisition per date

In [ ]:
go.Bar()

In [50]:
insider.columns

Index(['symbol', 'filingDate', 'transactionDate', 'reportingCik',
       'transactionType', 'securitiesOwned', 'companyCik', 'reportingName',
       'typeOfOwner', 'acquistionOrDisposition', 'formType',
       'securitiesTransacted', 'price', 'securityName', 'link'],
      dtype='object')

In [52]:
insider.transactionType.unique()

array(['G-Gift', 'S-Sale', 'M-Exempt', 'F-InKind', 'A-Award'],
      dtype=object)

In [53]:
insider.reportingName.unique()

array(['BELL JAMES A', 'Adams Katherine L.', "O'BRIEN DEIRDRE",
       'KONDO CHRIS', 'WILLIAMS JEFFREY E', 'Maestri Luca',
       'WAGNER SUSAN', 'SUGAR RONALD D', 'LOZANO MONICA C',
       'LEVINSON ARTHUR D', 'GORE ALBERT JR', 'Gorsky Alex',
       'JUNG ANDREA', 'COOK TIMOTHY D', 'IGER ROBERT A',
       'AHRENDTS ANGELA J', 'Srouji Johny', 'FEDERIGHI CRAIG',
       'Riccio Daniel J.', 'SCHILLER PHILIP W', 'Cue Eduardo H',
       'SEWELL D BRUCE'], dtype=object)

In [54]:
insider.typeOfOwner.unique()

array(['director', 'officer: SVP, GC and Secretary',
       'officer: Senior Vice President',
       'officer: Principal Accounting Officer', 'officer: COO',
       'officer: Senior Vice President, CFO',
       'director, officer: Chief Executive Officer',
       'officer: SVP, General Counsel',
       "officer: SVP, Gen'l Counsel, Secretary"], dtype=object)

In [60]:
insider[['reportingName', 'typeOfOwner']].drop_duplicates().sort_values(by='reportingName')

,reportingName,typeOfOwner
22,AHRENDTS ANGELA J,officer: Senior Vice President
1,Adams Katherine L.,"officer: SVP, GC and Secretary"
85,Adams Katherine L.,"officer: SVP, General Counsel"
0,BELL JAMES A,director
5,COOK TIMOTHY D,"director, officer: Chief Executive Officer"
46,Cue Eduardo H,officer: Senior Vice President
59,FEDERIGHI CRAIG,officer: Senior Vice President
45,GORE ALBERT JR,director
46,Gorsky Alex,director
92,IGER ROBERT A,director


In [55]:
insider.acquistionOrDisposition.unique()

array(['D', 'A'], dtype=object)

In [56]:
insider.formType.unique()

array(['4'], dtype=object)

In [57]:
insider.securityName.unique()

array(['Common Stock', 'Restricted Stock Unit',
       'Director Stock Option (Right to Buy)', 'Restricted Stock Units'],
      dtype=object)